In [1]:
cd "/Users/kz_ke/Documents/Masters/Classes/DL/AutoML3"

C:\Users\kz_ke\Documents\Masters\Classes\DL\AutoML3


In [ ]:
Time series forecast group 1 under security selection
Time series forecast group 1 under asset allocation

In [2]:
import q_learner as q_learner
import state_string_utils as stringutils
import state_enumerator as stateenum
import NAS 
import netparser
from tensorflow import keras
import numpy as np

In [3]:
_model = __import__('models.mnist',
                    globals(),
                    locals(),
                    ['state_space_parameters', 'hyper_parameters'], 
                    0)

In [4]:
factory = NAS.NAS('mylogs',
                  _model.state_space_parameters,
                  _model.hyper_parameters,
                  1,
                  1)

In [5]:
img_size = 28
n_classes = 10

#MNIST data image of shape 28*28=784
input_size = 784

# 0-9 digits recognition (labels)
output_size = 10

to_categorical = keras.utils.to_categorical 

#loading mnist dataset and seperating into train, validation and test sets
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)


x_train = np.reshape(x_train, [x_train.shape[0],28,28,1])
x_test = np.reshape(x_test, [x_test.shape[0],28,28,1])

validatationPct = 0.1
n = int(x_train.shape[0] * (1-validatationPct))
x_valid = x_train[n:]
x_train = x_train[:n]
#
y_valid = y_train[n:]
y_train = y_train[:n]

In [10]:
net, i = factory.generate_new_netork()
print(net)
p= netparser.parse('net', net)
newnet = netparser.parse_network_structure(p)
model = keras.Sequential(newnet)

callback = keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', patience=3, restore_best_weights = True)
model.compile(
            optimizer=keras.optimizers.Adam(lr=1e-4),
            loss='categorical_crossentropy',
            metrics=[keras.metrics.CategoricalAccuracy()]
        )

history = model.fit(x_train, y_train, batch_size = 40, epochs =1, callbacks=[callback], validation_data=(x_valid, y_valid))

bestval = model.evaluate(x_test, y_test)[1]

factory.incorporate_trained_net(net, bestval, 1, [i])

[C(128,1,1), P(2,2), D(1,2), C(64,5,1), SM(10)]
313/313 [==============================] - 9s 28ms/step - loss: 0.2905 - categorical_accuracy: 0.9167
None


Traceback (most recent call last):
  File "C:\Users\kz_ke\Documents\Masters\Classes\DL\AutoML3\NAS.py", line 143, in incorporate_trained_net
    self.qlearner.sample_replay_for_update()
  File "C:\Users\kz_ke\Documents\Masters\Classes\DL\AutoML3\q_learner.py", line 237, in sample_replay_for_update
    self.update_q_value_sequence(state_list, self.accuracy_to_reward(accuracy_best_val))
  File "C:\Users\kz_ke\Documents\Masters\Classes\DL\AutoML3\q_learner.py", line 245, in update_q_value_sequence
    self._update_q_value(states[-2], states[-1], termination_reward)
  File "C:\Users\kz_ke\Documents\Masters\Classes\DL\AutoML3\q_learner.py", line 265, in _update_q_value
    values[actions.index(action_between_states)] = values[actions.index(action_between_states)] + \
ValueError: <state_enumerator.State object at 0x0000017A58948F10> is not in list


In [30]:
states = factory.qlearner.stringutils.convert_model_string_to_states(netparser.parse('net', net))
states = factory.qlearner.stringutils.remove_drop_out_states(states)
bucket = [factory.qlearner.enum.bucket_state(state) for state in states]
bucket[3].as_tuple()

('conv', 3, 256, 5, 1, 8, 0, 0)

In [20]:
net, i = factory.generate_new_netork()
net

'[GAP(10), SM(10)]'

In [21]:
p= netparser.parse('net', net)
newnet = netparser.parse_network_structure(p)

In [22]:
newnet

In [25]:
model = keras.Sequential(newnet)

In [28]:
model.compile(
            optimizer=keras.optimizers.Adam(lr=1e-4),
            loss='categorical_crossentropy',
            metrics=[keras.metrics.CategoricalAccuracy()]
        )

In [33]:
model.fit(x_train, y_train, batch_size = 40, epochs =10)

Epoch 1/10
1500/1500 [==============================] - 8s 4ms/step - loss: 2.3073 - categorical_accuracy: 0.1124
Epoch 2/10
1500/1500 [==============================] - 5s 4ms/step - loss: 2.3055 - categorical_accuracy: 0.1124
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 2.3039 - categorical_accuracy: 0.1131
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 2.3025 - categorical_accuracy: 0.1141
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 2.3012 - categorical_accuracy: 0.1167
Epoch 6/10
1500/1500 [==============================] - 3s 2ms/step - loss: 2.3000 - categorical_accuracy: 0.1205
Epoch 7/10
1500/1500 [==============================] - 4s 3ms/step - loss: 2.2987 - categorical_accuracy: 0.1231
Epoch 8/10
1500/1500 [==============================] - 4s 3ms/step - loss: 2.2975 - categorical_accuracy: 0.1259
Epoch 9/10
1500/1500 [==============================] - 3s 2ms/step - loss: 2.2963 - cat

In [35]:
bestval = model.evaluate(x_test, y_test)[1]

313/313 [==============================] - 1s 3ms/step - loss: 2.2943 - categorical_accuracy: 0.1379


In [53]:
bestval

0.9860000014305115

In [37]:
net

'[GAP(10), SM(10)]'

In [47]:
factory.incorporate_trained_net(net, bestval, 1, [1])

Incorporated net, acc: 0.137900, net: [GAP(10), SM(10)]


In [49]:
factory.replay_dictionary

,net,accuracy_best_val,epsilon,time_finished
0,"[GAP(10), SM(10)]",0.1379,1,1.637442e+09


In [46]:
net

'[GAP(10), SM(10)]'

In [65]:
x_train.shape


(60000, 28, 28, 1)